# Kafka bridge to DFaaS

## select the right server and namespace

In [105]:
#ls ~/k8sconfig
export KUBECONFIG=~/k8sconfig/azure-ubuntu.conf
kubectl get nodes 

NAME     STATUS   ROLES                  AGE   VERSION
ubuntu   Ready    control-plane,master   38d   v1.25.6+k3s1


# Deployment

In [127]:
cat << EOF | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: kafka
  namespace: default
spec:
  ports:
  - name: kafka-9092
    nodePort: 30754
    port: 9092
  selector:
    app: kafka
  type: NodePort

---
apiVersion: apps/v1 
kind: Deployment
metadata:
  name: kafka-tunnel
spec:
  selector:
    matchLabels:
      app: kafka-tunnel
  strategy:
    type: Recreate
  template:
    metadata:
      labels:
        app: kafka-tunnel
    spec:

      containers:
      - image: registry.mydirk.de/dderichswei/kafkaingress:latest
        name: kafka-tunnel
        command: ["/bin/sh"]
        args: ["-c", "ssh -i /root/id_rsa_greenlake.pem dfaasusr@16.103.6.85 -L 9092:localhost:9092 -N -f && sleep infinity"]
        securityContext:
          privileged: true    
        ports:
        - name: kafka-9092
          containerPort: 9092
          
EOF


deployment.apps/kafka-tunnel-2 created
The Service "kafka-2" is invalid: spec.ports[0].nodePort: Invalid value: 30754: provided port is already allocated


: 1

In [126]:
kubectl get pods -o wide
kubectl get services -o wide

NAME                            READY   STATUS    RESTARTS       AGE   IP            NODE     NOMINATED NODE   READINESS GATES
my-release-postgresql-0         1/1     Running   17 (19d ago)   38d   10.42.0.157   ubuntu   <none>           <none>
mongodb-5c4769bdb-78qpn         1/1     Running   17 (19d ago)   38d   10.42.0.155   ubuntu   <none>           <none>
mariadb-0                       1/1     Running   14 (19d ago)   37d   10.42.0.160   ubuntu   <none>           <none>
mysql-1-0                       1/1     Running   10 (19d ago)   33d   10.42.0.161   ubuntu   <none>           <none>
kafka-tunnel-6b46dcf8d4-kxl6k   1/1     Running   0              31m   10.42.0.169   ubuntu   <none>           <none>
NAME                       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)           AGE   SELECTOR
kubernetes                 ClusterIP   10.43.0.1       <none>        443/TCP           38d   <none>
my-release-postgresql-hl   ClusterIP   None            <none>        5432/TCP    